In [2]:
#!py -m pip freeze > requirements.txt

In [2]:
from langchain import hub
from langchain_ollama.llms import OllamaLLM
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.callbacks import StdOutCallbackHandler
import requests

import os

In [4]:
# from langchain.globals import set_debug

# set_debug(True)


In [18]:
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]
#os.environ["TAVILY_API_KEY"]

In [4]:
# Use llama3.1 and test if it is working
model = OllamaLLM(model="llama3.1", temperature=0)
model.invoke("Come up with 10 names for a song about parrots")

'Here are 10 potential song title ideas about parrots:\n\n1. **"Flock Together"** - a catchy, upbeat title that captures the social nature of parrots.\n2. **"Rainbow Wings"** - inspired by the vibrant colors and stunning plumage of many parrot species.\n3. **"Squawkin\' Good Time"** - a fun, playful title that evokes the lively sounds made by parrots.\n4. **"Paradise Found"** - a title that references the tropical habitats where many parrots live.\n5. **"Feathered Friends"** - a heartwarming title that highlights the special bond between humans and parrots.\n6. **"The Parrot\'s Lament"** - a more melancholic title that suggests a song with a deeper, more emotional message.\n7. **"Winged Wonders"** - a title that celebrates the incredible abilities of parrots in flight.\n8. **"Colorful Chaos"** - a fun, whimsical title that captures the playful energy of parrots.\n9. **"The Bird\'s Eye View"** - a clever title that references both the physical perspective of a bird and the idea of seein

In [7]:
from langchain.agents import tool


@tool()
def get_number_active_cpf_members_and_employers(input: str) -> str:
    """Gets the number of all, Central Provident Fund (CPF) Members & Active Employers"""

    dataset_dict = {
        "active_member": "d_89dd4e5486fe3bb3b0c620cc6f787cdc",
        "active_employer": "d_bccb6829056e74f8a87b99c1bdb3e3ab",
        "member": "d_0014a7b5fa44981567b4c99c7a126630"
    }

    # Find out what type of question the user is asking
    question_classification_chain = model.invoke(f"""You are a chatbot tasked to identify if the following user query is asking for the 
                 1) Number of Central Provident Fund (CPF) active members. If user is asking for this, respond with active_member
                 2) Number of Central Provident Fund (CPF) active employers. If user is asking for this, respond with active_employer
                 3) Number of Central Provident Fund (CPF) members. If user is asking for this, respond with member

                 If you do not have the answer respond with 'I don't know'.
                                
                Query: {input}""")
    
    if question_classification_chain not in dataset_dict:
        return "I don't know"
    else:
        datasetId = dataset_dict[question_classification_chain]

    # Find out what year and quarter the user is asking for
    year_quarter_chain = model.invoke(f"""You are a chatbot tasked to identify the year and quarter from the user query 
                 1) If you do not know the year reply 2024
                 2) If you do not know the quarter reply Q1

                 Format your answer in the followng format YYYY-QQ. Only reply with the YYYY-QQ.
                                
                Query: {input}""")

    year_quarter_chain = year_quarter_chain.strip()
     
    url = "https://data.gov.sg/api/action/datastore_search"
    response = requests.get(url, params={"resource_id": datasetId})

    result = "I don't know"
    for i in response.json()['result']['records']:
        if i['qtr'] == year_quarter_chain:
            result = i[list(i.keys())[2]]
    

    return f"\nObservation: {result}"


@tool()
def web_search(input: str) -> str:
    """Runs web search for generic Central Provident Fund (CPF) related questions"""
    web_search_tool = TavilySearchResults()

    try:
        docs = web_search_tool.invoke({"query": input})
    except Exception as e:
        return f"Something went wrong using Tavily, {e}"
    

    output = model.invoke(f"""You are text summarization tool that will ONLY help to summarize relevant information.
        Use following piece of context to answer the question. 
        If you don't know the answer or there is no relevant context, just say you don't know. 
        Keep the answer within 5 sentences and concise.

        Context: {docs[0:3]}
        Question: {input}
        Answer: 

        """)

    
    return f"\nObservation: {output}"


@tool(return_direct=True)
def send_email_to_CPF(input: str) -> str:
    """Use this tool to send an email to Central Provident Fund (CPF) if the user is reporting a scam, giving a compliment or if you are unable to answer questions after using the web_search tool"""

    message_type_chain = model.invoke(f"You are a chatbot tasked with identifying the type of message that the user wrote. The type of message can be a Scam Report, Request, Appeal, Compliment, Feedback or Others. \n===CONVERSATION=== USER: {input}. MESSAGE_TYPE:"
                            )

    output = model.invoke(f"You are a chatbot tasked with writing an email to Central Provident Fund (CPF). Please come up with an email message based on the user's query.\n MESSAGE TYPE: {message_type_chain} ===CONVERSATION=== USER: {input}. EMAIL:"
                            )
    return output


@tool(return_direct=True)
def no_answer(input: str) -> str:
    """Come up with a generic fallback response when you get asked a question not related to Central Provident Fund (CPF)"""

    output = model.invoke(f"You are a chatbot that does not have sufficient information to answer the user's question. Please come up with a fallback response apologising to the user that you were not able to answer their question.\n===CONVERSATION=== USER: {input}. CHATBOT:"
                            )
    return output


tools = [
    get_number_active_cpf_members_and_employers,
    web_search,
    no_answer,
    send_email_to_CPF
]

#get_number_active_cpf_members_and_employers.invoke("how many active cpf members are there?")
#web_search.invoke("what is CPF?")
#send_email_to_CPF.invoke("What is CPF?")

In [21]:
prompt = hub.pull("hwchase17/react")
print(prompt.template)

In [9]:
prompt.template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Thought: you should always think about what to do. If you have enough information, give your Final Answer. If you don't know, use the no_answer tool.
Action: the action to take, should be one of [{tool_names}]
Action Input: the Action Input is the EXACT verbatim of the Question asked. Do not add or change the Question.
Observation: Additional information given by the tool that can be used in the Final Answer.

This Thought/Action/Action Input/Observation can repeat if you do not have enough information.

Once you have enough information to answer the user question, do the following:
Thought: I now know the final answer
Final Answer: ONLY give your final answer to the question that was asked.

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [10]:
memory = ChatMessageHistory(session_id="test-session")

In [11]:
agent = create_react_agent(model, tools, prompt)

handler = StdOutCallbackHandler()
agent_executor = AgentExecutor(agent=agent, tools=tools, max_iterations=3, verbose=True, return_intermediate_steps=True, callbacks=[handler])


agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [12]:
agent_executor.invoke(
    {"input": "How old do I need to be to claim CPF?"}
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I'm not sure about the age requirement for claiming CPF. I should try to find more information.
Action: web_search
Action Input: What is the minimum age required to claim Central Provident Fund (CPF
Observation: I don't know. There is no relevant information provided in the context about the minimum age required to claim the Central Provident Fund (CPF). The context only discusses the eligibility criteria for CPF contributions, but not the minimum age for claiming benefits.Here's my response:

Thought: I'm still unsure about the age requirement for claiming CPF. Let me try another approach.
Action: send_email_to_CPF
Action Input: I am unable to answer questions related to Central Provident Fund (CPF) after using web_search tool, can you please provide more information on how old one needs to be to claim CPFHere's a draft email message based on the user's query:

Subject: Request for Information on CPF Claim Age

Dear CPF Team,

I am writing to seek your assistance in understan

{'input': 'How old do I need to be to claim CPF?',
 'output': "Here's a draft email message based on the user's query:\n\nSubject: Request for Information on CPF Claim Age\n\nDear CPF Team,\n\nI am writing to seek your assistance in understanding the requirements for claiming my Central Provident Fund (CPF) benefits. I was unable to find the information I needed through online searches, and I would appreciate it if you could provide me with more details.\n\nSpecifically, I would like to know what is the minimum age requirement to claim CPF benefits? Could you please provide me with the relevant information or point me in the direction of a resource where I can find this information?\n\nThank you for your time and assistance. I look forward to hearing from you soon.\n\nBest regards,\n[Your Name]",
 'intermediate_steps': [(AgentAction(tool='web_search', tool_input='What is the minimum age required to claim Central Provident Fund (CPF', log="Thought: I'm not sure about the age requirement

In [13]:
agent_executor.invoke(
    {"input": "What is CPF?"}
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I'm not sure what CPF stands for or what it means. I should try to find more information.
Action: web_search
Action Input: What is CPF
Observation: The Central Provident Fund (CPF) is a mandatory social security savings scheme in Singapore that helps meet retirement, housing, and healthcare needs. It's funded by employer and employee contributions. The government also supplements lower wage workers' CPF savings through various schemes. There are three main accounts within the CPF: Ordinary, Special, and MediSave, each with different uses and benefits.Thought: I now know that CPF stands for Central Provident Fund, which is a mandatory social security savings scheme in Singapore.

Final Answer: The Central Provident Fund (CPF) is a mandatory social security savings scheme in Singapore that helps meet retirement, housing, and healthcare needs. It's funded by employer and employee contributions.

> Finished chain.


{'input': 'What is CPF?',
 'output': "The Central Provident Fund (CPF) is a mandatory social security savings scheme in Singapore that helps meet retirement, housing, and healthcare needs. It's funded by employer and employee contributions.",
 'intermediate_steps': [(AgentAction(tool='web_search', tool_input='What is CPF', log="Thought: I'm not sure what CPF stands for or what it means. I should try to find more information.\nAction: web_search\nAction Input: What is CPF"),
   "\nObservation: The Central Provident Fund (CPF) is a mandatory social security savings scheme in Singapore that helps meet retirement, housing, and healthcare needs. It's funded by employer and employee contributions. The government also supplements lower wage workers' CPF savings through various schemes. There are three main accounts within the CPF: Ordinary, Special, and MediSave, each with different uses and benefits.")]}

In [14]:
agent_executor.invoke(
    {"input": "How much CPF do employers need to contribute?"}
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I can use the web_search tool to find information about employer contributions to CPF.
Action: web_search
Action Input: What are the employer contribution rates for Central Provident Fund (CPF
Observation: The employer's CPF contribution rate is 17% for those up to the age of 55, and decreases to 7.5% for those aged 70 and above. However, it can also be 20% as mentioned in another source. The total CPF contribution rate is 37% (if monthly wage >$750) with employer contributing 17%.Thought: I now know the final answer
Final Answer: The employer's CPF contribution rate is 17% for those up to the age of 55, and decreases to 7.5% for those aged 70 and above. However, it can also be 20% as mentioned in another source.

> Finished chain.


{'input': 'How much CPF do employers need to contribute?',
 'output': "The employer's CPF contribution rate is 17% for those up to the age of 55, and decreases to 7.5% for those aged 70 and above. However, it can also be 20% as mentioned in another source.",
 'intermediate_steps': [(AgentAction(tool='web_search', tool_input='What are the employer contribution rates for Central Provident Fund (CPF', log='Thought: I can use the web_search tool to find information about employer contributions to CPF.\nAction: web_search\nAction Input: What are the employer contribution rates for Central Provident Fund (CPF'),
   "\nObservation: The employer's CPF contribution rate is 17% for those up to the age of 55, and decreases to 7.5% for those aged 70 and above. However, it can also be 20% as mentioned in another source. The total CPF contribution rate is 37% (if monthly wage >$750) with employer contributing 17%.")]}

In [15]:
agent_executor.invoke(
    {"input": "What is the weather in Singapore tommorow?"}
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: This question doesn't seem related to Central Provident Fund (CPF). If I don't have enough information, I should use the no_answer tool. However, since this question can be answered using general knowledge or a web search, I'll try to find more information first.

Action: web_search
Action Input: What is the weather in Singapore tomorrow
Observation: I don't know. There is no information about the weather forecast for tomorrow in the provided context. The context only provides general information about the average temperature and rainfall in Singapore during September, but does not include a specific forecast for the next day.Thought: Since I couldn't find any relevant information about the weather forecast for tomorrow using web search, it's likely that this question is not related to Central Provident Fund (CPF). In that case, I should use the no_answer tool.

Action: no_answer
Action Input: What is the weather in Singapore tomorrowI'm so sorry, but I don't have access to re

{'input': 'What is the weather in Singapore tommorow?',
 'output': "I'm so sorry, but I don't have access to real-time weather information or specific forecasts for locations like Singapore. My training data only allows me to provide general information and responses that are not location-specific. I'd be happy to try and help with something else, though! Would you like to know more about a different topic?",
 'intermediate_steps': [(AgentAction(tool='web_search', tool_input='What is the weather in Singapore tomorrow', log="Thought: This question doesn't seem related to Central Provident Fund (CPF). If I don't have enough information, I should use the no_answer tool. However, since this question can be answered using general knowledge or a web search, I'll try to find more information first.\n\nAction: web_search\nAction Input: What is the weather in Singapore tomorrow"),
   "\nObservation: I don't know. There is no information about the weather forecast for tomorrow in the provided co

In [16]:
agent_executor.invoke(
    {"input": "Thank you so much for helping me answer CPF related questions!"}
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: You're welcome! It's my pleasure to assist with Central Provident Fund (CPF) related queries. I'm ready to help!

Action: send_email_to_CPF
Action Input: Thank you so much for helping me answer CPF related questionsHere's the email message based on the user's query:

Subject: Appreciation for Assistance with CPF Queries

Dear Central Provident Fund (CPF) Team,

I am writing to express my sincere gratitude and appreciation for your assistance in answering my queries regarding CPF-related matters. Your prompt and helpful responses have been invaluable, and I am truly thankful for the support provided.

Your team's dedication to providing accurate and timely information has made a significant difference in helping me understand and navigate the CPF system. Please pass on my thanks to the entire team involved in assisting me.

Thank you again for your excellent service!

Best regards,
[Your Name]


> Finished chain.


{'input': 'Thank you so much for helping me answer CPF related questions!',
 'output': "Here's the email message based on the user's query:\n\nSubject: Appreciation for Assistance with CPF Queries\n\nDear Central Provident Fund (CPF) Team,\n\nI am writing to express my sincere gratitude and appreciation for your assistance in answering my queries regarding CPF-related matters. Your prompt and helpful responses have been invaluable, and I am truly thankful for the support provided.\n\nYour team's dedication to providing accurate and timely information has made a significant difference in helping me understand and navigate the CPF system. Please pass on my thanks to the entire team involved in assisting me.\n\nThank you again for your excellent service!\n\nBest regards,\n[Your Name]",
 'intermediate_steps': [(AgentAction(tool='send_email_to_CPF', tool_input='Thank you so much for helping me answer CPF related questions', log="Thought: You're welcome! It's my pleasure to assist with Centr

In [17]:
agent_with_chat_history.invoke(
        {"input": "How many CPF active users are there?"},
        config={"configurable": {"session_id": "<foo>"}},
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I can use the get_number_active_cpf_members_and_employers tool to find out. If not, I'll try web_search for more information.
Action: get_number_active_cpf_members_and_employers
Action Input: How many CPF active users are there
Observation: 2160000Thought: I now know the final answer.

Final Answer: 2,160,000.

> Finished chain.


{'input': 'How many CPF active users are there?',
 'chat_history': [],
 'output': '2,160,000.',
 'intermediate_steps': [(AgentAction(tool='get_number_active_cpf_members_and_employers', tool_input='How many CPF active users are there', log="Thought: I can use the get_number_active_cpf_members_and_employers tool to find out. If not, I'll try web_search for more information.\nAction: get_number_active_cpf_members_and_employers\nAction Input: How many CPF active users are there"),
   '\nObservation: 2160000')]}